<a href="https://colab.research.google.com/github/SyedaMeem/cifar10/blob/main/CIFAR10_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CIFAR10->airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck.
They were collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton.
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images

The CIFAR-100 dataset ->
This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).

Here is the list of some classes in the CIFAR-100:

Superclass ->        Classes
[aquatic mammals	->   beaver, dolphin, otter, seal, whale].
[fish ->	             aquarium fish, flatfish, ray, shark, trout].
[flowers	->           orchids, poppies, roses, sunflowers, tulips].

In [ ]:
import tensorflow as tf

In [16]:
from tensorflow import keras
from tensorflow.keras import layers,regularizers
from tensorflow.keras.datasets import cifar10

In [ ]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

170500096/170498071 [==============================] - 3s 0us/step
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


(50000,32,32,3)->(no. of images,each image width,each image height,rgb color model-3 channnel)

In [ ]:
#converting float64->float32 to minimize computation
#divide by 255 gray level value to normalize all pixels values between 0 to 1

x_train=x_train.astype("float32")/255.0
x_test=x_test.astype("float32")/255.0


#Sequential API

In [ ]:
##Convolutional Network

#padding='valid' -> the output of the layer according to kernel operation i.e. may be 30*30. it is set by default,so need not to mention it explicitly.
#padding='same' -> the output of the layer according to prvious layer i.e. input layer 32*32. it is not set by default,so need to mention it explicitly.

model=keras.Sequential(
    [
     keras.Input(shape=(32,32,3)),
     layers.Conv2D(32,(3,3),padding='valid',activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64,(3,3),padding='valid',activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(128,(3,3),padding='valid',activation='relu'),
     layers.Flatten(),
     layers.Dense(64,activation='relu'),
     layers.Dense(10),
    ]
)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [ ]:
##Training  network configuration
# from_logits=True ->it's going to send the softmax function to the output layer of the network 
#and then it's going to map it to sparse categorical cross entropy

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"],
)

In [ ]:
#More of the concrete training of that network

model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)
model.evaluate(x_test,y_test,batch_size=64,verbose=2)


Epoch 1/10
782/782 - 60s - loss: 1.7128 - accuracy: 0.3765
Epoch 2/10
782/782 - 59s - loss: 1.3492 - accuracy: 0.5207
Epoch 3/10
782/782 - 59s - loss: 1.2193 - accuracy: 0.5695
Epoch 4/10
782/782 - 59s - loss: 1.1258 - accuracy: 0.6069
Epoch 5/10
782/782 - 59s - loss: 1.0510 - accuracy: 0.6350
Epoch 6/10
782/782 - 60s - loss: 0.9837 - accuracy: 0.6582
Epoch 7/10
782/782 - 60s - loss: 0.9332 - accuracy: 0.6760
Epoch 8/10
782/782 - 59s - loss: 0.8834 - accuracy: 0.6965
Epoch 9/10
782/782 - 60s - loss: 0.8433 - accuracy: 0.7075
Epoch 10/10
782/782 - 60s - loss: 0.8027 - accuracy: 0.7230
157/157 - 3s - loss: 0.8892 - accuracy: 0.6952


[0.8892107605934143, 0.6952000260353088]

#Functional API


In [ ]:
##Convolutional Network

def my_model():
  inputs = keras.Input(shape=(32,32,3))
  x=layers.Conv2D(32,3)(inputs)
  x=layers.BatchNormalization()(x)
  x=keras.activations.relu(x)
  x=layers.MaxPooling2D()(x)
  x=layers.Conv2D(64,5,padding='same')(x)
  x=layers.BatchNormalization()(x)
  x=keras.activations.relu(x)
  x=layers.Conv2D(128,3,padding='valid')(x)
  x=layers.BatchNormalization()(x)
  x=keras.activations.relu(x)
  x=layers.Flatten()(x)
  x=layers.Dense(64,activation='relu')(x)
  outputs=layers.Dense(10)(x)
  model=keras.Model(inputs=inputs,outputs=outputs)
  return model

model=my_model()

In [ ]:
##Convolutional Network with regularizer and dropout

def my_model():
  inputs = keras.Input(shape=(32,32,3))
  x=layers.Conv2D(32,3,padding='same',kernel_regularizer=regularizers.l2(0.01),)(inputs)
  x=layers.BatchNormalization()(x)
  x=keras.activations.relu(x)
  x=layers.MaxPooling2D()(x)
  x=layers.Conv2D(64,5,padding='same',kernel_regularizer=regularizers.l2(0.01),)(x)
  x=layers.BatchNormalization()(x)
  x=keras.activations.relu(x)
  x=layers.Conv2D(128,3,padding='same',kernel_regularizer=regularizers.l2(0.01),)(x)
  x=layers.BatchNormalization()(x)
  x=keras.activations.relu(x)
  x=layers.Flatten()(x)
  x=layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01),)(x)
  x=layers.Dropout(0.5)(x)
  outputs=layers.Dense(10)(x)
  model=keras.Model(inputs=inputs,outputs=outputs)
  return model

model=my_model()

In [15]:
##Training  network configuration
# from_logits=True ->it's going to send the softmax function to the output layer of the network 
#and then it's going to map it to sparse categorical cross entropy

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"],
)

In [ ]:
#More of the concrete training of that network

model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)
model.evaluate(x_test,y_test,batch_size=64,verbose=2)


Epoch 1/10
782/782 - 264s - loss: 1.2842 - accuracy: 0.5453
Epoch 2/10
782/782 - 254s - loss: 0.8795 - accuracy: 0.6926
Epoch 3/10
782/782 - 266s - loss: 0.7235 - accuracy: 0.7459
Epoch 4/10
782/782 - 288s - loss: 0.6039 - accuracy: 0.7891
Epoch 5/10
782/782 - 277s - loss: 0.5154 - accuracy: 0.8202
Epoch 6/10
782/782 - 248s - loss: 0.4266 - accuracy: 0.8530
Epoch 7/10
782/782 - 252s - loss: 0.3557 - accuracy: 0.8777
Epoch 8/10
782/782 - 247s - loss: 0.2954 - accuracy: 0.8989
Epoch 9/10
782/782 - 245s - loss: 0.2343 - accuracy: 0.9218
Epoch 10/10
782/782 - 245s - loss: 0.1903 - accuracy: 0.9373
157/157 - 12s - loss: 1.0825 - accuracy: 0.7021


[1.0824909210205078, 0.7020999789237976]